In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv("train.csv")

In [30]:
test=pd.read_csv("test.csv")

In [4]:
train['text'].dtype

dtype('O')

In [66]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.wrappers import Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.models import model_from_json
from keras_tqdm import TQDMNotebookCallback

In [6]:
#a=train['text'][0]

In [7]:
#b=a.split(' ')

In [9]:
d=[]
for i in range(train.shape[0]):
    a=train['text'][i]
    b=a.split(' ')
    c=[]
    for a in range(len(b)):
        c.append(int(b[a]))
    d.append(c)    

In [10]:
#d=np.array(d)

In [11]:
train_X=d[:12000]
train_Y=train['author'][:12000]
val_X=d[12001:]
val_Y=train['author'][12001:]

In [31]:
d=[]
for i in range(test.shape[0]):
    a=test['text'][i]
    b=a.split(' ')
    c=[]
    for a in range(len(b)):
        c.append(int(b[a]))
    d.append(c)    
    
#d=np.array(d) 
test_X=d[:]

In [13]:
author_to_index = {
    "EAP": 0, 
    "HPL": 1, 
    "MWS": 2
}
train_Y = train_Y.map(author_to_index)
val_Y = val_Y.map(author_to_index)


author_to_index = {
    "EAP": 0, 
    "HPL": 1, 
    "MWS": 2
}
train_Y = train_Y.map(author_to_index)
val_Y = val_Y.map(author_to_index)


In [14]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
train_Y_hot = pd.get_dummies(train_Y)
train_Y_hot = train_Y_hot.values
type(train_Y)
val_Y_hot = pd.get_dummies(val_Y)
val_Y_hot = val_Y_hot.values

In [15]:
m=0
for i in range(len(train_X)):
    m=m+len(train_X[i])
m=m/len(train_X)
print(m)        

51.10391666666667


In [39]:
max_length = 20
train_X = sequence.pad_sequences(train_X, maxlen=max_length)
val_X = sequence.pad_sequences(val_X, maxlen=max_length)

In [67]:
# create the model 3

model = Sequential()
model.add(Embedding(1000, 100, input_length=max_length))

model.add(Conv1D(filters=500, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(BatchNormalization())
model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, activation='relu'))
#model.add(Dropout(0.2))

model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, activation='relu'))

#model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, activation='relu'))

model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, activation='relu'))
#model.add(Dropout(0.2))

#model.add(Dropout(0.2))
model.add(LSTM(512))

#model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())
#model.fit(train_X, train_Y, validation_data=(val_X, val_Y), epochs=3, batch_size=64)
model.fit(train_X, train_Y_hot, validation_data=(val_X, val_Y_hot),epochs=5, batch_size=32, verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True)])

/home/dj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="softmax", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 20, 100)           100000    
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 20, 500)           150500    
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 6, 500)            0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 6, 500)            2000      
_________________________________________________________________
lstm_31 (LSTM)               (None, 6, 512)            2074624   
_________________________________________________________________
lstm_32 (LSTM)               (None, 6, 512)            2099200   
_________________________________________________________________
lstm_33 (LSTM)               (None, 6, 512)            2099200   
__________

In [64]:
scores = model.evaluate(train_X, train_Y_hot, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 82.14%


In [54]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [58]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(train_X, train_Y_hot, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 71.09%


In [42]:
test_X = sequence.pad_sequences(test_X, maxlen=max_length)

In [62]:
p=model.predict(test_X)

In [44]:
p.shape

(8392, 3)

In [63]:
np.savetxt("sub4.csv", p, delimiter=",")

In [ ]:
# create the model

model = Sequential()
model.add(Embedding(1000, 100, input_length=max_length))
model.add(Conv1D(filters=500, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(BatchNormalization())
model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, activation='relu'))
#model.add(Dropout(0.2))

model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, activation='relu'))

#model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, activation='relu'))

model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, activation='relu'))
#model.add(Dropout(0.2))

#model.add(Dropout(0.2))
model.add(LSTM(512))

#model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax', bias_regularizer='regularizers.l2(0.01)'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())
#model.fit(train_X, train_Y, validation_data=(val_X, val_Y), epochs=3, batch_size=64)
model.fit(train_X, train_Y_hot, validation_data=(val_X, val_Y_hot), epochs=5, batch_size=32, verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True)])